In [ ]:
!pip install keras

In [ ]:
!pip install --upgrade adversarial-robustness-toolbox

Requirement already up-to-date: adversarial-robustness-toolbox in /usr/local/lib/python3.7/dist-packages (1.7.0)


In [ ]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Activation, GlobalAveragePooling2D, MaxPooling2D
from keras import regularizers, initializers

# total params: 0.27M in vgg-20

momentum = 0.9
epsilon = 1e-5
weight_decay = 1e-4


def conv_bn_relu(x, filters, name, kernel_size=(3, 3), strides=(1, 1)):
    """conv2D + batch normalization + relu activation"""

    x = Conv2D(
        filters, kernel_size,
        strides=strides, padding='same', use_bias=False,
        kernel_initializer=initializers.he_normal(),
        kernel_regularizer=regularizers.l2(weight_decay),
        name=name + '_conv2D'
    )(x)
    x = BatchNormalization(momentum=momentum, epsilon=epsilon, name=name + '_BN')(x)
    x = Activation('relu', name=name + '_relu')(x)
    return x


def conv_blocks(x, filters, num_blocks, name):
    """two conv, downsampling if dimension not match"""

    for i in range(num_blocks):
        if int(x.shape[-1]) != filters:
            x = conv_bn_relu(x, filters, strides=(2, 2), name=name + '_blk{}_conv1'.format(i + 1))
        else:
            x = conv_bn_relu(x, filters, name + '_blk{}_conv1'.format(i + 1))
        x = conv_bn_relu(x, filters, name + '_blk{}_conv2'.format(i + 1))
    return x


def vgg_model(x, num_classes, num_blocks):
    """sequential model without shortcut, same number of parameters as its resnet counterpart"""

    # level 0:
    # input: 32x32x3; output: 32x32x16
    x = conv_bn_relu(x, 16, name='lv0')

    # level 1:
    # input: 32x32x16; output: 32x32x16
    x = conv_blocks(x, 16, num_blocks, name='lv1')

    # level 2:
    # input: 32x32x16; output: 16x16x32
    x = conv_blocks(x, 32, num_blocks, name='lv2')

    # level 3:
    # input: 16x16x32; output: 8x8x64
    x = conv_blocks(x, 64, num_blocks, name='lv3')

    # output
    x = GlobalAveragePooling2D(name='global_pool')(x)
    x = Dense(
        num_classes,
        activation='softmax',
        kernel_initializer=initializers.he_normal(),
        kernel_regularizer=regularizers.l2(weight_decay),
        name='FC'
    )(x)
    return x

In [ ]:
from keras.datasets import cifar10


from keras.layers import Input
import numpy as np
import random
import tensorflow as tf

from art.classifiers import KerasClassifier
from art.attacks import TargetedUniversalPerturbation
from art.utils import random_sphere

from keras import utils, Model
from tensorflow.keras.utils import to_categorical

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
norm2_mean = 0
for im in x_train:
    norm2_mean += np.linalg.norm(im.flatten(), ord=2)
norm2_mean /= len(x_train)

# normalize data
channel_mean = np.mean(x_train, axis=(0, 1, 2))
channel_std = np.std(x_train, axis=(0, 1, 2))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
for i in range(3):
    x_train[:, :, :, i] = (x_train[:, :, :, i] - channel_mean[i]) / channel_std[i]
    x_test[:, :, :, i] = (x_test[:, :, :, i] - channel_mean[i]) / channel_std[i]

# labels to categorical
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# randomly selecting 1000 images for each class in 10 classes 
trainIdx = []
num_each_class = 10
for class_i in range(10):
    np.random.seed(111)
    idx = np.random.choice( np.where(y_train[:, class_i]==1)[0], num_each_class, replace=False ).tolist()
    trainIdx = trainIdx + idx
random.shuffle(trainIdx)
x_train, y_train = x_train[trainIdx], y_train[trainIdx]

# build vgg model
num_blocks = 3
img_input = Input(shape=(32, 32, 3), name='input')
img_prediction = vgg_model(img_input, num_classes, num_blocks)
model = Model(img_input, img_prediction)
model.load_weights("vgg_20_1553198546.h5")

# build targeted UAP


tf.compat.v1.disable_eager_execution()


classifier = KerasClassifier(model=model)
adv_crafter = TargetedUniversalPerturbation(
    classifier,
    attacker='fgsm',
    delta=0.000001,
    attacker_params={'targeted':True, 'eps':0.006},
    max_iter=10,
    eps=5.5,
    norm=2)

# set target label
target = 1
y_train_adv_tar = np.zeros(y_train.shape)
for i in range(y_train.shape[0]):
    y_train_adv_tar[i, target] = 1.0

# generate noise
_ = adv_crafter.generate(x_train, y=y_train_adv_tar)
noise = adv_crafter.noise[0,:]
norm2_ori = np.linalg.norm(noise.flatten(), ord=2)

# targeted UAP result
print('=== Targeted UAP ===')
rescaled_noise = noise.copy()
for i in range(3):
    rescaled_noise[:, :, i] = rescaled_noise[:, :, i] * channel_std[i]
norm2 = np.linalg.norm(rescaled_noise.flatten(), ord=2)
normInf = np.abs(rescaled_noise.flatten()).max()
print('norm2: {:.1f} %'.format(norm2/norm2_mean*100))

x_train_adv = x_train + noise
x_test_adv = x_test + noise

preds_train_adv = np.argmax(classifier.predict(x_train_adv), axis=1)
preds_test_adv = np.argmax(classifier.predict(x_test_adv), axis=1) 
targeted_success_rate_train = np.sum(preds_train_adv == target) / len(x_train)
targeted_success_rate_test = np.sum(preds_test_adv == target) / len(x_test) 
print('targeted_success_rate_train: {:.1f} %'.format(targeted_success_rate_train*100))
print('targeted_success_rate_test: {:.1f} %'.format(targeted_success_rate_test*100))

np.save('noise.npy', noise)

# random noise result 
print('=== Random Noise ===')
rescaled_noise_rand = random_sphere(nb_points=1,nb_dims=(32*32*3),radius=norm2_ori,norm=2)
rescaled_noise_rand = rescaled_noise_rand.reshape(32,32,3)
noise_rand = rescaled_noise_rand.copy()
for i in range(3):
    noise_rand[:, :, i] = noise_rand[:, :, i] * channel_std[i]
norm2_rand = np.linalg.norm(noise_rand.flatten(), ord=2)
normInf_rand = np.abs(noise_rand.flatten()).max()
print('norm2_rand: {:.1f} %'.format(norm2_rand/norm2_mean*100))

x_train_adv_rand = x_train + noise_rand
x_test_adv_rand = x_test + noise_rand

preds_train_adv_rand = np.argmax(classifier.predict(x_train_adv_rand), axis=1)
preds_test_adv_rand = np.argmax(classifier.predict(x_test_adv_rand), axis=1)
targeted_success_rate_train_rand = np.sum(preds_train_adv_rand == target) / len(x_train)
targeted_success_rate_test_rand = np.sum(preds_test_adv_rand == target) / len(x_test)
print('targeted_success_rate_train_rand: {:.1f} %'.format(targeted_success_rate_train_rand*100))
print('targeted_success_rate_test_rand: {:.1f} %'.format(targeted_success_rate_test_rand*100))

ImportError: ignored

In [ ]:
# Loading the dataset

from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels)= cifar10.load_data()

# Labels before applying the function
# Training set labels
print(train_labels)
print(train_labels.shape)

# Testing set labels
print(test_labels)
print(test_labels.shape)

# Applying the function to training set labels and testing set labels
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels, dtype ="uint8")
test_labels = to_categorical(test_labels, dtype ="uint8")

# Labels after applying the function
# Training set labels
print(train_labels)
print(train_labels.shape)

# Testing set labels
print(test_labels)
print(test_labels.shape)


170508288/170498071 [==============================] - 11s 0us/step
[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]
(50000, 1)
[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]
(10000, 1)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
(50000, 10)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
(10000, 10)


In [ ]:
!pip  install tensorflow 

In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np


from keras.models import Sequential


from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowClassifier
from art.utils import load_mnist

# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 2: Create the model

tf.compat.v1.disable_eager_execution()



input_ph = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
labels_ph = tf.placeholder(tf.int32, shape=[None, 10])

x = tf.layers.conv2d(input_ph, filters=4, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.conv2d(x, filters=10, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.flatten(x)
x = tf.layers.dense(x, 100, activation=tf.nn.relu)
logits = tf.layers.dense(x, 10)

loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels_ph))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

sess.run(tf.global_variables_initializer())

# Step 3: Create the ART classifier

classifier = TensorFlowClassifier(
    clip_values=(min_pixel_value, max_pixel_value),
    input_ph=input_ph,
    output=logits,
    labels_ph=labels_ph,
    train=train,
    loss=loss,
    learning=None,
    sess=sess,
    preprocessing_defences=[],
)

# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)

# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is depreca

AttributeError: ignored

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.5.0
